In [1]:
import json
import requests
import time

In [2]:
json_path = 'data/attestations.json'
with open(json_path, 'r') as f:
    data = json.load(f)

In [3]:
def fetch_attestations(schema_id, time_created_after=0):

    url = 'https://optimism.easscan.org/graphql'
    query_limit = 100

    query = '''
    query Attestations($schemaId: StringFilter!, $skip: Int!, $take: Int!, $timeCreatedAfter: IntFilter) {
        attestations(where: {schemaId: $schemaId, timeCreated: $timeCreatedAfter}, take: $take, skip: $skip) {
            id
            attester
            recipient
            refUID
            revocable
            revocationTime
            expirationTime
            timeCreated 
            decodedDataJson    
        }
    }
    '''
    
    variables = {
        "schemaId": {
            "equals": schema_id
        },
        "skip": 0,
        "take": query_limit,
        "timeCreatedAfter": {
            "gt": time_created_after
        },
    }

    headers = {
        'Content-Type': 'application/json',
    }

    all_attestations = []

    while True:
        payload = {
            'query': query,
            'variables': variables
        }

        try:
            response = requests.post(url, headers=headers, data=json.dumps(payload))
            response.raise_for_status()

            data = response.json()
            attestations = data.get('data', {}).get('attestations', [])
            all_attestations.extend(attestations)

            if len(attestations) < query_limit:
                break

            variables["skip"] += query_limit

        except (requests.exceptions.RequestException, json.JSONDecodeError) as e:
            print(f"Failed to fetch attestations for {schema_id}: {str(e)}")
            break

    print(f"Total attestations for Schema ID {schema_id}: {len(all_attestations)}")
    return all_attestations

In [4]:
attestations = fetch_attestations('0xc9bc703e3c48be23c1c09e2f58b2b6657e42d8794d2008e3738b4ab0e2a3a8b6')

Total attestations for Schema ID 0xc9bc703e3c48be23c1c09e2f58b2b6657e42d8794d2008e3738b4ab0e2a3a8b6: 149


In [5]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
    'Referer': 'https://metricsgarden.xyz'
}

def get_metadata(uri, tries=0):
    if tries == 3:
        print("Too many tries.")
        return

    response = requests.get(uri, headers=headers)
    if response.status_code == 200:
        return response.json()
    
    if response.status_code == 429:
        time.sleep(180)
        return get_metadata(uri, tries=tries+1)
    
    else:
        print(f"Error {response.status} at {uri}")
        return

    
def process_attestation(a):
    attestation_data = {
        'id': a['id'],
        'attester': a['attester'],
        'recipient': a['recipient'],
        'timeCreated': a['timeCreated']
    }
    decoded_json = json.loads(a['decodedDataJson'])
    for obj in decoded_json:
        obj_name = obj['name']
        obj_value = obj['value']['value']
        attestation_data.update({obj_name:obj_value})
        if obj_name == 'metadataurl':
            print(obj_value)
            metadata = get_metadata(obj_value)
            if not metadata:
                return
            attestation_data.update({'metadata': metadata})
    return attestation_data

In [7]:
idx = len(data)
if idx < len(attestations):
    for a in attestations[idx:]:
        if a['revocationTime']:
            continue
        attestation_data = process_attestation(a)
        if attestation_data:
            data.append(attestation_data)
            
    with open(json_path, "w") as f:
        json.dump(data, f, indent=2)